In [1]:
#pip install ipywidgets

In [2]:
import gradio as gr
import os
from zipfile import ZipFile
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from keras import layers
import numpy as np

IMG_SIZE = 256  # Updated to 256

# Function to load the data and get classes
def load_data():
    # Extract the data from the zip file
    data_path = 'lung_image_sets.zip'
    with ZipFile(data_path, 'r') as zip:
        zip.extractall()
    print('The dataset has been extracted.')
    # Get the list of classes
    path = 'lung_image_sets'
    classes = os.listdir(path)
    return classes

# Create the model architecture to match your original model
def create_model():
    model = keras.models.Sequential([
        layers.Conv2D(filters=32, kernel_size=(5, 5), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3), padding='same'),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D(2, 2),
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.BatchNormalization(),
        layers.Dense(3, activation='softmax')
    ])
    return model

# Load the classes and model
classes = load_data()
model = create_model()

# Print model summary
#model.summary()

# Load pre-trained weights
weights_path = 'weights/weights.keras'  # Update this path to your weight file
if os.path.exists(weights_path):
    print("Loading pre-trained weights...")
    try:
        model.load_weights(weights_path)
        print("Weights loaded successfully!")
    except Exception as e:
        print(f"Error loading weights: {e}")
        print("Model expects the following shapes:")
        for layer in model.layers:
            if hasattr(layer, 'input_shape') and hasattr(layer, 'output_shape'):
                print(f"{layer.name}: Input shape = {layer.input_shape}, Output shape = {layer.output_shape}")
            else:
                print(f"{layer.name}: Shape information not available")
else:
    raise FileNotFoundError(f"Weight file not found at {weights_path}")

# Function to classify the image
def classify_image(image):
    image = image.resize((IMG_SIZE, IMG_SIZE))
    image_array = tf.keras.preprocessing.image.img_to_array(image)
    image_array = tf.expand_dims(image_array, 0) / 255.0  # Normalize

    prediction = model.predict(image_array)
    predicted_class = classes[np.argmax(prediction)]
    confidence = np.max(prediction)
    
    return f"{predicted_class} (Confidence: {confidence:.2f})"

# Gradio interface
def lung_cancer_app():
    with gr.Blocks() as demo:
        gr.Markdown("# Lung Cancer Classification")
        with gr.Column():
            image_input = gr.Image(type="pil", label="Input Image")
            classify_button = gr.Button("Classify")
        with gr.Row():
            prediction_output = gr.Label(label="Prediction")
        
        classify_button.click(
            fn=classify_image,
            inputs=image_input,
            outputs=prediction_output
        )
    demo.launch()

if __name__ == "__main__":
    lung_cancer_app()

The dataset has been extracted.
Loading pre-trained weights...


C:\Users\POOJA DEORE\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Weights loaded successfully!
Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step
